In [3]:
from pyrep.vrep.vrep import simxGetFloatSignal
from pyrep.vrep import vrep as v

class lineTracer:

    def __init__(self, api: VRep):
        self._api = api
        self._left_motor = api.joint.with_velocity_control("DynamicLeftJoint")
        self._right_motor = api.joint.with_velocity_control("DynamicRightJoint")
        self._left_sensor = api.sensor.vision("LeftSensor")
        self._right_sensor = api.sensor.vision("RightSensor")
        self.id = api._id
    
    #The following function will return the total accumulated angle turned by the left wheel
    def getval(self):
        return simxGetFloatSignal(self.id,'leftEncoder',v.simx_opmode_streaming)#simx_opmode_streaming, simx_opmode_buffer)

    #The following function will return the total accumulated rotation count of the left wheel
    def getrots(self):
        return simxGetFloatSignal(self.id,'leftEncoder_rots',v.simx_opmode_streaming)#simx_opmode_buffer)

    def stop(self):
        self._set_two_motor(0, 0)
        
    def fwd_right(self, speed=20.0):
        self._set_two_motor(speed, speed/2)

    def fwd_left(self, speed=20.0):
        self._set_two_motor(speed/2, speed)

    def move_forward(self, speed=20.0):
        self._set_two_motor(speed, speed)

    def move_backward(self, speed=20.0):
        self._set_two_motor(-speed, -speed)

    def _set_two_motor(self, left: float, right: float):
        self._left_motor.set_target_velocity(left)
        self._right_motor.set_target_velocity(right)
    
    def right_line(self):
        return self._left_sensor.read()[0]
    
    def left_line(self):
        return self._left_sensor.read()[0]

    def position_left_joint(self):
        return self._left_motor.get_position(), 